In [1]:
import ctypes as C

In [2]:
bucket = C.CDLL('./src/BucketSort/BucketSort.so')
bucket.bucket_sort.restype = C.POINTER(C.c_uint64)
bucket.bucket_sort.argtypes = [C.POINTER(C.c_uint64), C.c_uint64, ]

radix = C.CDLL('./src/RadixSort/RadixSort.so')
radix.radix_sort.restype = C.POINTER(C.c_uint64)
radix.radix_sort.argtypes = [C.POINTER(C.c_uint64), C.c_uint64, ]

quick = C.CDLL('./src/QuickSort/QuickSort.so')
quick.c_quick_sort.restype = C.POINTER(C.c_uint64)
quick.c_quick_sort.argtypes = [C.POINTER(C.c_uint64), C.c_uint64, ]

In [3]:
# поразрядная сортировка
# возвращает итоговую перестановку эелементов массива A
# для того, чтобы можно было переставить элементы в любом другом массиве
def radix_argsort(A):
    arr = (C.c_uint64 * len(A))(*A)
    res = radix.radix_sort(C.cast(C.byref(arr), C.POINTER(C.c_uint64)), len(A))
    return [res[i] for i in range(len(A))]

In [4]:
# карманная сортировка
# возвращает итоговую перестановку эелементов массива A
# для того, чтобы можно было переставить элементы в любом другом массиве
def bucket_argsort(A):
    arr = (C.c_uint64 * len(A))(*A)
    res = bucket.bucket_sort(C.cast(C.byref(arr), C.POINTER(C.c_uint64)), len(A))
    return [res[i] for i in range(len(A))]

In [5]:
# быстрая сортировка
# возвращает итоговую перестановку эелементов массива A
# для того, чтобы можно было переставить элементы в любом другом массиве
def quick_argsort(A):
    arr = (C.c_uint64 * len(A))(*A)
    res = quick.c_quick_sort(C.cast(C.byref(arr), C.POINTER(C.c_uint64)), len(A))
    return [res[i] for i in range(len(A))]

In [6]:
# детерминированный алгоритм для поиска мостов
# на вход поступает граф представленный списком смежности
# саписок представлен как словарь(хеш-таблица) списков
# выход представляет собой список ребер, являющихся мостами 
#def compute_bridges_determ(adj_list):
#    adj_list_char_str = adj_list_to_byte(adj_list)
#    result = bridge.compute_bridges_determ(adj_list_char_str, len(adj_list_char_str))
#    return result.decode('utf-8')

class DetermBridges:
    def __init__(self):
        self.timer = 0
        self.history = []
        self.bridges = []
        self.colors = []
        self.M = []
        self.entry = []
        self.adj_list = {}
    
    def bridges_determ_dfs(self, vertex):
        self.colors[vertex] = 'gray'
        self.history.append(vertex)
        self.timer += 1
        self.entry[vertex] = self.timer
        for adjacent in self.adj_list[vertex]:
            if (self.colors[adjacent] == 'white'):
                self.bridges_determ_dfs(adjacent)
                self.M[vertex] = min(self.M[vertex], self.M[adjacent])
                if (self.M[adjacent] > self.entry[vertex]):
                    if (vertex < adjacent):
                        self.bridges.append((vertex, adjacent))
                    else:
                        self.bridges.append((adjacent, vertex))
            elif len(self.history) < 2 or self.history[-2] != adjacent:
                self.M[vertex] = min(self.M[vertex], self.entry[adjacent])
        self.colors[vertex] = 'black'
        self.history.pop()

    def determ_bridges(self, adj_list):
        self.timer = 0
        self.history = []
        self.bridges = []
        self.colors = ['white' for i in adj_list]
        self.M = [float('inf') for i in adj_list]
        self.entry = [0 for i in adj_list]
        self.adj_list = adj_list
        for i in adj_list:
            if self.colors[i] == 'white':
                self.bridges_determ_dfs(i)
        return self.bridges
    
def compute_bridges_determ(adj_list):
    determ = DetermBridges()
    return determ.determ_bridges(adj_list)

In [7]:
import numpy as np
import random as rd

class RandomBridges:

    def __init__(self):
        self.adj_list = {}
        self.bridges = []
        self.edges = []
        self.colors = []
        self.history = []
        self.samples = {}
        self.EDGE = 0
        self.SAMPLE = 1

    def sampling_dfs(self, vertex):
        self.colors[vertex] = 'gray'
        self.history.append(vertex)
        for adjacent in self.adj_list[vertex]:
            if self.colors[adjacent] == 'white':
                self.sampling_dfs(adjacent)
            elif len(self.history) < 2 or self.history[-2] != adjacent:
                rand = np.uint64(rd.randint(0, np.iinfo(np.uint64).max))
                self.samples[vertex][adjacent] = rand;
                self.samples[adjacent][vertex] = rand;
        self.colors[vertex] = 'black';
        self.history.pop()

        if len(self.history) > 0:
            parent = self.history[-1]
            parent_edge_weight = np.uint64(0)
            for adjacent in self.adj_list[vertex]:
                if (adjacent != parent):
                    parent_edge_weight ^= self.samples[vertex][adjacent]
            self.samples[vertex][parent] = parent_edge_weight
            self.samples[parent][vertex] = parent_edge_weight
        
        
    def launch_sampling(self):
        self.colors = ['white' for i in self.adj_list]
        for vertex in self.adj_list:
            if self.colors[vertex] == 'white':
                self.sampling_dfs(vertex)
        
    def find_bridges(self, adj_list):
        self.adj_list = adj_list
        for i in range(len(adj_list)):
            self.samples[i] = {}
        self.launch_sampling()
        for first_key in self.adj_list:
            for second_key in self.adj_list[first_key]:
                if (second_key, first_key) not in self.bridges and self.samples[first_key][second_key] == 0:
                    self.bridges.append((first_key, second_key))
        return self.bridges
    
    def find_2bridges(self, adj_list, sort_fun):
        self.adj_list = adj_list
        for i in range(len(adj_list)):
            self.samples[i] = {}
        self.launch_sampling()
        for first_key in self.adj_list:
            for second_key in self.adj_list[first_key]:
                if (second_key, first_key) not in [edge[self.EDGE] for edge in self.edges]:
                    self.edges.append([(first_key, second_key), self.samples[first_key][second_key]])
        samples_list = [edge[self.SAMPLE] for edge in self.edges]
        sorted_args = sort_fun(samples_list)
        cluster_size = 0
        current_cluster = 0
        for i in range(len(sorted_args) - 1):
            cluster_size+=1
            if self.edges[sorted_args[i]][self.SAMPLE] != self.edges[sorted_args[i+1]][self.SAMPLE]:
                if (cluster_size > 1):
                    self.bridges.append([])
                    j = i - cluster_size + 1
                    while (j < i):
                        self.bridges[current_cluster].append((self.edges[sorted_args[j]][self.EDGE]))
                        j+=1
                    cluster_size = 0
                    current_cluster+=1
        return self.bridges

In [8]:
# рандомизированный алгоритм для поиска мостов
# на вход поступает граф представленный списком смежности
# саписок представлен как словарь(хеш-таблица) списков 
# выход представляет собой список ребер, являющихся мостами с большой вероятностью
def compute_bridges_rand(adj_list):
    rand = RandomBridges()
    return rand.find_bridges(adj_list)

In [9]:
# рандомизированный алгоритм для поиска 2-мостов
#
# на вход поступает граф представленный списком смежности и алгоритм сортировки для меток на ребрах
# саписок представлен как словарь(хеш-таблица) списков 
#
# выходом алгоритма является список списков ребер
# в каждом списке любая пара ребер должна с высокой вероятностью образовывать 2-мост
# например, если выходом является спискок [[e1, e2, e3],[e4, e5]]
# то с высокой вероятностью 2-мостами будут пары ребер: (e1,e2), (e1,e3), (e2,e3), (e4,e5)
# ребра здесь это пары вершин типа e1 = (1,2)
def compute_2bridges_rand(adj_list, sort_fun):
    rand = RandomBridges()
    return rand.find_2bridges(adj_list, sort_fun)

In [10]:
def DFS_recursive(graph, S, colors, black_order):
    colors[S] = 'g'
    
    for adjacent in graph[S]:
        if (colors[adjacent] == 'w'):
            DFS_recursive(graph, adjacent, colors, black_order)
            
    colors[S] = 'b'
    black_order.append(S)
    
    
def DFS_wrapper(graph):
    colors = []
    black_order = []
    
    for i in range(len(graph)):
        colors.append('w')
    DFS_recursive(graph, next(iter(graph)), colors, black_order)
    return black_order
    
    
def convert_to_edge_graph(graph):
    edges_list = []
    
    for vertex in graph:
        for adjacent in graph[vertex]:
            edges_list.append((vertex, adjacent))
            
    edge_graph = {}
    
    for edge in range(len(edges_list)):
        edge_graph[edge] = []
        for potential_adjacent in range(len(edges_list)):
            if edges_list[edge][1] == edges_list[potential_adjacent][0]:
                edge_graph[edge].append(potential_adjacent)
                
    return edges_list, edge_graph
# функция вычисляющая эйлеров обход для данного ориентированного графа G 
# цикл кодируется списком ребер, например:
# (1,2), (2,4), (4, 7), (7,1)
#
def compute_Euler_circuit_digraph(adj_list):
  # тут должен быть Ваш код
  #G = nx.DiGraph(adj_list)
  #return list(nx.algorithms.eulerian_circuit(G))
    graph = adj_list
    edges_list, edge_graph = convert_to_edge_graph(graph)
    cycle = []
    for edge_index in DFS_wrapper(edge_graph):
        cycle.insert(0, edges_list[edge_index])
    return cycle

# функция вычисляющая эйлеров обход для данного простого графа G 
# цикл кодируется списком ребер, например:
# (1,2), (2,4), (4, 7), (7,1)
#
#def compute_Euler_circuit_simple(adj_list):
#  # тут должен быть Ваш код
#  G = nx.Graph(adj_list)
#  return list(nx.algorithms.eulerian_circuit(G))

In [11]:
invert = C.CDLL('./so/Invert.so')

invert.invert_in_Zp_Euclead.restype = C.c_uint64
invert.invert_in_Zp_Euclead.argtypes = [C.c_uint64, C.c_uint64, ]

invert.invert_in_Zp_Ferma.restype = C.c_uint64
invert.invert_in_Zp_Ferma.argtypes = [C.c_uint64, C.c_uint64, ]

In [12]:
# эта функция инвертирует x в Zp с помощью алгоритма Евклида
# выходом является целое число y, такое что
# 1) 0 < y < p 
# 2) (xy) mod p = 1 
# если число невозможно инвертировать, функция возвращает 0
def invert_in_Zp_Euclead(p,x):
    return invert.invert_in_Zp_Euclead(p, x)

In [13]:
# эта функция инвертирует x в Zp с помощью малой теоремы Ферма и алгоритма быстрого возведения в степень
# выходом является целое число y, такое что
# 1) 0 < y < p 
# 2) (xy) mod p = 1 
# если число невозможно инвертировать, функция возвращает 0
def invert_in_Zp_Ferma(p,x):
    return invert.invert_in_Zp_Ferma(p, x)